# preprocessing for movies data frame

# recomandation with content based 

In [30]:
#import the relevant libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#storing the data from (movies and ratings) csv file into data frame:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')




#using regex and extract number with paranthesis from column title and add it to a new column called year
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)

#using regex to extract the parenthesis fro the year column
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)

#using regex to relplace the year in the title column with nothing
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')

#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

#spliting the genres column in movies data farme based on |
movies_df['genres'] = movies_df.genres.str.split('|')

#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
moviesWithGenres_df = movies_df.copy()

#for each index in a row get all the datas and for each genres in genres column ,put 1 at point index and that genres
#that has been chosen.since some of the cells will be missed so we will fill the missing daa with 0
#so the movie  doesnt have ttaht column's genre
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

#now we cleaned the movies data frame

<ipython-input-30-2d1e7ac86a3b>:20: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
ratings_df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [31]:
#from rating data frame remove the time stamp column since we dont need it here
ratings_df = ratings_df.drop('timestamp', 1)



we have a data frame called inputMovies that has all the rates that the user gave to certain movies

we also know the genres of those movies , now we give it some movies with its genres and calculate that how will the 

user rate these movies.

We multiply **input users rating** matrices and **movies matrix** and then we have **weighted genre matrix**.

then sum each column score normilize it then called the matrix **User Profile**.

then writing the genres of those movies that the user havent rate or seen.

now We multiply matrices **User profile** and **new movies matrix** and then we have **weighted genre matrix** for 

movies he hasnt watch at all.
 
then sum each row of the weighted genres matrix and called  **Recomandation Matrix**

In [34]:
#having the rate for some movies that the user had seen , 
#create it in a dictionary and then transform it to the dataframe named inputMovies
inputMovies = pd.DataFrame({'title':['Breakfast Club, The','Toy Story','Jumanji','Pulp Fiction','Akira'],'rating':[5,3.5,2,5,4.5]})


#from movies data farme give all the column for just movies in inputmovies so we can have the movie id of those movies
#that the user had rated.
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]

inputId

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
257,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994
973,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988
1445,1968,"Breakfast Club, The","[Comedy, Drama]",1985


In [37]:
#mergeng the filtered data frame and the inputmovies with each other so we cav have the ratings of the movies
inputMovies = pd.merge(inputId, inputMovies)

#sicne we no longer need to use genres and year column we can drop'em.
inputMovies = inputMovies.drop(['genres','year'],1)

#filter the movieswithgenres data frame so we have all tthe matrix for genres of the movies that the user rated.
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]

#reset the index
userMovies = userMovies.reset_index(drop=True)

#drop column we wont be using now we have the genres matrix
userGenreTable = userMovies.drop(['movieId','title','genres','year'], 1)

#creating user profile by multipling 'input users rating' matrices and 'genres' matrix
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])




In [42]:
#puting the movie id column as index
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])

#drop columns we dont need
genreTable = genreTable.drop(['movieId','title','genres','year'],1)


#mulipy the genres matrix (genreTable) and user profile and calculate the sum of each row.
#now we have the movie id of the movies we have to suggest to the user.
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())

#sorting the recomandation
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)

#we have to find the name of the movie id in movies_df from recommandationTable 
#now we have 20 movies that we can recomand the user
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]


,movieId,title,genres,year
559,673,Space Jam,"[Adventure, Animation, Children, Comedy, Fanta...",1996
1390,1907,Mulan,"[Adventure, Animation, Children, Comedy, Drama...",1998
2250,2987,Who Framed Roger Rabbit?,"[Adventure, Animation, Children, Comedy, Crime...",1988
3460,4719,Osmosis Jones,"[Action, Animation, Comedy, Crime, Drama, Roma...",2001
4631,6902,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",2002
5490,26340,"Twelve Tasks of Asterix, The (Les douze travau...","[Action, Adventure, Animation, Children, Comed...",1976
5819,32031,Robots,"[Adventure, Animation, Children, Comedy, Fanta...",2005
6047,40339,Chicken Little,"[Action, Adventure, Animation, Children, Comed...",2005
6448,51939,TMNT (Teenage Mutant Ninja Turtles),"[Action, Adventure, Animation, Children, Comed...",2007
6455,52287,Meet the Robinsons,"[Action, Adventure, Animation, Children, Comed...",2007
